<a href="https://colab.research.google.com/github/Jaehyeon-kr/SeoulTech_AI_Class/blob/main/%5BAI2025%5D_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Space

In [3]:
import os
import random

In [4]:
path = "cdn.cs50.net/ai/2020/spring/lectures/3/src3/hospitals/assets/"
os.system("wget -rkpN -np -e robots=off path " + path)

1024

In [5]:
class Space():

    def __init__(self, height, width, num_hospitals):
        """Create a new state space with given dimensions."""
        self.height = height
        self.width = width
        self.num_hospitals = num_hospitals
        self.houses = set()
        self.hospitals = set()

    def add_house(self, row, col):
        """Add a house at a particular location in state space."""
        self.houses.add((row, col))

    def available_spaces(self):
        """Returns all cells not currently used by a house or hospital."""

        # Consider all possible cells
        candidates = set(
            (row, col)
            for row in range(self.height)
            for col in range(self.width)
        )

        # Remove all houses and hospitals
        for house in self.houses:
            candidates.remove(house)
        for hospital in self.hospitals:
            candidates.remove(hospital)
        return candidates

    def hill_climb(self, maximum=None, image_prefix=None, log=False):
        """Performs hill-climbing to find a solution."""
        count = 0

        # Start by initializing hospitals randomly
        self.hospitals = set()
        for i in range(self.num_hospitals):
            self.hospitals.add(random.choice(list(self.available_spaces())))
        if log:
            print("Initial state: cost", self.get_cost(self.hospitals))
        if image_prefix:
            self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

        # Continue until we reach maximum number of iterations
        while maximum is None or count < maximum:
            count += 1
            best_neighbors = []
            best_neighbor_cost = None

            # Consider all hospitals to move
            for hospital in self.hospitals:

                # Consider all neighbors for that hospital
                for replacement in self.get_neighbors(*hospital):

                    # Generate a neighboring set of hospitals
                    neighbor = self.hospitals.copy()
                    neighbor.remove(hospital)
                    neighbor.add(replacement)

                    # Check if neighbor is best so far
                    cost = self.get_cost(neighbor)
                    if best_neighbor_cost is None or cost < best_neighbor_cost:
                        best_neighbor_cost = cost
                        best_neighbors = [neighbor]
                    elif best_neighbor_cost == cost:
                        best_neighbors.append(neighbor)

            # None of the neighbors are better than the current state
            if best_neighbor_cost >= self.get_cost(self.hospitals):
                return self.hospitals

            # Move to a highest-valued neighbor
            else:
                if log:
                    print(f"Found better neighbor: cost {best_neighbor_cost}")
                self.hospitals = random.choice(best_neighbors)

            # Generate image
            if image_prefix:
                self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

    def random_restart(self, maximum, image_prefix=None, log=False):
        """Repeats hill-climbing multiple times."""
        best_hospitals = None
        best_cost = None

        # Repeat hill-climbing a fixed number of times
        for i in range(maximum):
            hospitals = self.hill_climb()
            cost = self.get_cost(hospitals)
            if best_cost is None or cost < best_cost:
                best_cost = cost
                best_hospitals = hospitals
                if log:
                    print(f"{i}: Found new best state: cost {cost}")
            else:
                if log:
                    print(f"{i}: Found state: cost {cost}")

            if image_prefix:
                self.output_image(f"{image_prefix}{str(i).zfill(3)}.png")

        return best_hospitals

    def get_cost(self, hospitals):
        """Calculates sum of distances from houses to nearest hospital."""
        cost = 0
        for house in self.houses:
            cost += min(
                abs(house[0] - hospital[0]) + abs(house[1] - hospital[1])
                for hospital in hospitals
            )
        return cost

    def get_neighbors(self, row, col):
        """Returns neighbors not already containing a house or hospital."""
        candidates = [
            (row - 1, col),
            (row + 1, col),
            (row, col - 1),
            (row, col + 1)
        ]
        neighbors = []
        for r, c in candidates:
            if (r, c) in self.houses or (r, c) in self.hospitals:
                continue
            if 0 <= r < self.height and 0 <= c < self.width:
                neighbors.append((r, c))
        return neighbors

    def output_image(self, filename):
        """Generates image with all houses and hospitals."""
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        cost_size = 40
        padding = 10

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "white"
        )
        house = Image.open(path + "images/House.png").resize(
            (cell_size, cell_size)
        )
        hospital = Image.open(path + "images/Hospital.png").resize(
            (cell_size, cell_size)
        )
        font = ImageFont.truetype(path + "fonts/OpenSans-Regular.ttf", 30)
        draw = ImageDraw.Draw(img)

        for i in range(self.height):
            for j in range(self.width):

                # Draw cell
                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                draw.rectangle(rect, fill="black")

                if (i, j) in self.houses:
                    img.paste(house, rect[0], house)
                if (i, j) in self.hospitals:
                    img.paste(hospital, rect[0], hospital)

        # Add cost
        draw.rectangle(
            (0, self.height * cell_size, self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "black"
        )
        draw.text(
            (padding, self.height * cell_size + padding),
            f"Cost: {self.get_cost(self.hospitals)}",
            fill="white",
            font=font
        )

        img.save(filename)

Create a new space and add houses randomly

In [6]:
s = Space(height=10, width=20, num_hospitals=3)
for i in range(15):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

Use local search to determine hospital placement

In [7]:
hospitals = s.hill_climb(image_prefix="hospitals", log=True)

Initial state: cost 100
Found better neighbor: cost 95
Found better neighbor: cost 90
Found better neighbor: cost 86
Found better neighbor: cost 82
Found better neighbor: cost 78
Found better neighbor: cost 76
Found better neighbor: cost 74
Found better neighbor: cost 72
Found better neighbor: cost 70
Found better neighbor: cost 69
Found better neighbor: cost 68
Found better neighbor: cost 67
Found better neighbor: cost 64
Found better neighbor: cost 63
Found better neighbor: cost 62
Found better neighbor: cost 61
Found better neighbor: cost 58
Found better neighbor: cost 55
Found better neighbor: cost 54
Found better neighbor: cost 53
Found better neighbor: cost 52
Found better neighbor: cost 51


Use random_restart search to determine hospital placement

In [8]:
hospitals = s.random_restart(20, image_prefix="hospitals", log=True)

0: Found new best state: cost 61
1: Found state: cost 82
2: Found new best state: cost 56
3: Found state: cost 64
4: Found new best state: cost 52
5: Found new best state: cost 51
6: Found state: cost 54
7: Found state: cost 72
8: Found state: cost 57
9: Found state: cost 57
10: Found state: cost 57
11: Found state: cost 56
12: Found state: cost 61
13: Found state: cost 57
14: Found state: cost 51
15: Found state: cost 62
16: Found state: cost 63
17: Found state: cost 62
18: Found state: cost 51
19: Found state: cost 52


# Linear Programming



* Objective Function: $50x_1 + 80x_2$
* Constraint 1: $5x_1 + 2x_2 <= 20$
* Constraint 2: $-10x_1 + -12x_2 <= -90$

In [9]:
import scipy.optimize

result = scipy.optimize.linprog(
    [50, 80],  # Cost function: 50x_1 + 80x_2
    A_ub=[[5, 2], [-10, -12]],  # Coefficients for inequalities
    b_ub=[20, -90],  # Constraints for inequalities: 20 and -90
)

if result.success:
    print(f"X1: {round(result.x[0], 2)} hours")
    print(f"X2: {round(result.x[1], 2)} hours")
else:
    print("No solution")


X1: 1.5 hours
X2: 6.25 hours


#Constconstraint Satisfaction

Naive backtracking search without any heuristics or inference.

In [10]:
VARIABLES = ["A", "B", "C", "D", "E", "F", "G"]
CONSTRAINTS = [
    ("A", "B"),
    ("A", "C"),
    ("B", "C"),
    ("B", "D"),
    ("B", "E"),
    ("C", "E"),
    ("C", "F"),
    ("D", "E"),
    ("E", "F"),
    ("E", "G"),
    ("F", "G")
]


def backtrack(assignment):
    """Runs backtracking search to find an assignment."""

    # Check if assignment is complete
    if len(assignment) == len(VARIABLES):
        return assignment

    # Try a new variable
    var = select_unassigned_variable(assignment)
    for value in ["Monday", "Tuesday", "Wednesday"]:
        new_assignment = assignment.copy()
        new_assignment[var] = value
        if consistent(new_assignment):
            result = backtrack(new_assignment)
            if result is not None:
                return result
    return None


def select_unassigned_variable(assignment):
    """Chooses a variable not yet assigned, in order."""
    for variable in VARIABLES:
        if variable not in assignment:
            return variable
    return None


def consistent(assignment):
    """Checks to see if an assignment is consistent."""
    for (x, y) in CONSTRAINTS:

        # Only consider arcs where both are assigned
        if x not in assignment or y not in assignment:
            continue

        # If both have same value, then not consistent
        if assignment[x] == assignment[y]:
            return False

    # If nothing inconsistent, then assignment is consistent
    return True

In [11]:
solution = backtrack(dict())
print(solution)

{'A': 'Monday', 'B': 'Tuesday', 'C': 'Wednesday', 'D': 'Wednesday', 'E': 'Monday', 'F': 'Tuesday', 'G': 'Wednesday'}


In [12]:
!pip install python-constraint

  Preparing metadata (setup.py) ... done
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24061 sha256=b4275bd0d9c7c5dc2e34453d3d96b99d07d0e53b09d6b55f801092b14c85ac40
  Stored in directory: /root/.cache/pip/wheels/c1/d2/3d/082849b61a9c6de02d4a7c8a402c224640f08d8a971307b92b
Successfully built python-constraint


In [13]:
from constraint import *

problem = Problem()

# Add variables
problem.addVariables(
    ["A", "B", "C", "D", "E", "F", "G"],
    ["Monday", "Tuesday", "Wednesday"]
)

# Add constraints
CONSTRAINTS = [
    ("A", "B"),
    ("A", "C"),
    ("B", "C"),
    ("B", "D"),
    ("B", "E"),
    ("C", "E"),
    ("C", "F"),
    ("D", "E"),
    ("E", "F"),
    ("E", "G"),
    ("F", "G")
]
for x, y in CONSTRAINTS:
    problem.addConstraint(lambda x, y: x != y, (x, y))


Solve problem

In [14]:
for solution in problem.getSolutions():
    print(solution)

{'E': 'Wednesday', 'B': 'Tuesday', 'C': 'Monday', 'F': 'Tuesday', 'A': 'Wednesday', 'D': 'Monday', 'G': 'Monday'}
{'E': 'Wednesday', 'B': 'Monday', 'C': 'Tuesday', 'F': 'Monday', 'A': 'Wednesday', 'D': 'Tuesday', 'G': 'Tuesday'}
{'E': 'Tuesday', 'B': 'Wednesday', 'C': 'Monday', 'F': 'Wednesday', 'A': 'Tuesday', 'D': 'Monday', 'G': 'Monday'}
{'E': 'Tuesday', 'B': 'Monday', 'C': 'Wednesday', 'F': 'Monday', 'A': 'Tuesday', 'D': 'Wednesday', 'G': 'Wednesday'}
{'E': 'Monday', 'B': 'Tuesday', 'C': 'Wednesday', 'F': 'Tuesday', 'A': 'Monday', 'D': 'Wednesday', 'G': 'Wednesday'}
{'E': 'Monday', 'B': 'Wednesday', 'C': 'Tuesday', 'F': 'Wednesday', 'A': 'Monday', 'D': 'Tuesday', 'G': 'Tuesday'}
